In [1]:
import pandas as pd
from collections import Counter

In [2]:
df = pd.read_json("../../data/OE-Adapt-Multipref.jsonl", lines=True)

In [3]:
df = df.rename(columns={"prompt_id": "prompt_hash", "completion_a": "response_a", "completion_b": "response_b"})

In [4]:
# Only get overall preference
def get_majority(row, preference="overall_pref", binarize=True):
    normal_workers = [i["overall_pref"] for i in row["normal_worker_annotations"]]
    expert_workers = [i["overall_pref"] for i in row["expert_worker_annotations"]]
    all_annotations = normal_workers + expert_workers

    if binarize:
        final_annotations = []
        for annotation in all_annotations:
            if annotation == "A-is-slightly-better":
                annotation = "A-is-clearly-better"
            elif annotation == "B-is-slightly-better":
                annotation = "B-is-clearly-better"
            else:
                annotation = annotation
            final_annotations.append(annotation)
    else:
        final_annotations = all_annotations


    return Counter(final_annotations).most_common()[0][0]

# Use majority preference as main human preference
df["pref_human"] = df.apply(lambda x: get_majority(x), axis=1)


In [5]:
df.sort_values(by="prompt_hash")

,comparison_id,prompt_hash,text,model_a,response_a,model_b,response_b,source,category,subject_study,highest_level_degree,normal_worker_annotations,expert_worker_annotations,pref_human
0,cdccd42baceb4358888fda8c23a7fb32,000494cd98ae43f58b0d57732a37e66a,but it will provide acceleration in y directio...,gpt-4-turbo-2024-04-09,To calculate the distance traveled in the y-di...,meta-llama/Meta-Llama-3-70B-Instruct,To find the distance traveled in the y-directi...,lmsys/sharegpt-vicuna,Open QA,Physics,Mathematics and statistics,"[{'helpful_pref': 'A-is-clearly-better', 'help...","[{'helpful_pref': 'A-is-clearly-better', 'help...",A-is-clearly-better
1,aa8af1e5ea024ad08e71273f5695993b,000616d1471540e4935f6c60ecc06aeb,what are some fun sandwiches i can make for kids?,allenai/tulu-2-7b,Here are a few fun sandwich ideas that kids mi...,meta-llama/Meta-Llama-3-70B-Instruct,"Kids love sandwiches, and making them fun can ...",anthropic/helpful-base,Brainstorm,Leisure,Arts & Humanities,"[{'helpful_pref': 'B-is-slightly-better', 'hel...","[{'helpful_pref': 'B-is-clearly-better', 'help...",B-is-clearly-better
2,6b9671fb535b447e898a24771d136306,000616d1471540e4935f6c60ecc06aeb,what are some fun sandwiches i can make for kids?,allenai/tulu-2-70b,Here are a few ideas for fun sandwiches that k...,gpt-4-turbo-2024-04-09,Making sandwiches for kids is not only about p...,anthropic/helpful-base,Brainstorm,Leisure,Arts & Humanities,"[{'helpful_pref': 'B-is-clearly-better', 'help...","[{'helpful_pref': 'B-is-clearly-better', 'help...",B-is-clearly-better
3,924dbf961d664b1c8634a0d831f0c8d4,00117e5830704b6a8d448a6400e3cf8e,what are august activiites for copenhagen rank...,allenai/tulu-2-70b,"I'm sorry, but I don't have information about ...",meta-llama/Llama-2-70b-chat-hf,Here are some popular activities to do in Co...,lmsys/sharegpt-vicuna,Open QA,Travel and Tourism,Journalism & Information Business,"[{'helpful_pref': 'B-is-clearly-better', 'help...","[{'helpful_pref': 'B-is-clearly-better', 'help...",B-is-clearly-better
4,fb0315fba0f1484994e2a9b3e7e1d4cb,00117e5830704b6a8d448a6400e3cf8e,what are august activiites for copenhagen rank...,allenai/tulu-2-7b,Here are some popular activities to do in Cope...,allenai/tulu-2-70b,"I'm sorry, but I don't have information about ...",lmsys/sharegpt-vicuna,Open QA,Travel and Tourism,Journalism & Information Business,"[{'helpful_pref': 'A-is-clearly-better', 'help...","[{'helpful_pref': 'A-is-clearly-better', 'help...",A-is-clearly-better
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7920,f1274570e13244d08cb4a219a25428c2,ffbdefdcb4aa4ace9f0d13ac52dfb707,Can I fly a drone into someones yard?,allenai/tulu-2-7b,"In general, it is not legal to fly a drone int...",allenai/tulu-2-70b,It is generally not advisable to fly a drone i...,anthropic/harmless-base,Open QA,Law,Administration & Law,"[{'helpful_pref': 'Tie', 'helpful_checked_reas...","[{'helpful_pref': 'A-is-slightly-better', 'hel...",A-is-clearly-better
7921,9693bb96c1d84322af9d3f6c0a2650ac,ffc4ab9208734efb8249a9edb73b35fe,I’m trying to find the distance between the su...,allenai/tulu-2-7b,The average distance between the Earth and the...,meta-llama/Meta-Llama-3-70B-Instruct,A classic question!\n\nThe average distance be...,anthropic/harmless-base,Open QA,Physics,Natural sciences,"[{'helpful_pref': 'B-is-slightly-better', 'hel...","[{'helpful_pref': 'B-is-slightly-better', 'hel...",B-is-clearly-better
7922,01cb90b78e844f4797b8344761774906,ffc4ab9208734efb8249a9edb73b35fe,I’m trying to find the distance between the su...,allenai/tulu-2-7b,The average distance between the Earth and the...,meta-llama/Llama-2-70b-chat-hf,The average distance between the sun and the...,anthropic/harmless-base,Open QA,Physics,Natural sciences,"[{'helpful_pref': 'Tie', 'helpful_checked_reas...","[{'helpful_pref': 'B-is-clearly-better', 'help...",B-is-clearly-better
7923,d3c2536329f04ccaa52b69d7c4b2fd34,ffd3af32d10a44cbaab471de41cf509d,What is the most controversial topic in t

In [6]:
gpt4_df = pd.read_json("../../data/OE-Adapt-GPT4-Overall.jsonl", lines=True).rename(columns={"pref": "pref_gpt4"})

In [8]:
df = df.merge(gpt4_df[["comparison_id", "pref_gpt4", "evaluator"]], on="comparison_id", how="left")

In [9]:
df.to_json("multipref_human_vs_gpt4_overall.jsonl", lines=True, orient="records")